In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import os

from toddler.action_coding import mass_answers, force_answers
from toddler.models import ValueNetwork
from toddler.RecurrentWorker import train
from toddler.validate import validate

from isaac.models import ComplexRNNModel

from toddler.simulator.config import generate_every_world_configuration, generate_cond

from generate_passive_simulations import get_configuration_answer

In [6]:
model_directory = "models/grab_a_puck/"
data_directory = "grab_a_puck_plots/"

In [7]:
from isaac.utils import get_cuda_device_if_available
device = get_cuda_device_if_available()
print(device)

cuda:0


In [8]:
discount_factor = 0.95

every_conf = generate_every_world_configuration()
every_world_answer = np.array(list(map(get_configuration_answer, every_conf)))
n_configurations = len(every_conf)

train_size = 0.7
val_size = 0.15
test_size = 0.15

all_indices = np.arange(n_configurations)
train_indices, not_train_indices = train_test_split(all_indices, train_size=train_size,
                                                    random_state=0, stratify=every_world_answer)
val_indices, test_indices = train_test_split(not_train_indices, train_size=0.5,                
                                             random_state=0,
                                             stratify=every_world_answer[not_train_indices])

N_WORLDS = 33000
timeout = 600
TOTAL_STEPS = min(N_WORLDS*timeout, np.inf)

print("N_WORLDS", N_WORLDS, "TOTAL_STEPS", TOTAL_STEPS)

torch.manual_seed(0)
np.random.seed(0)
repeated_train_indices = np.random.choice(train_indices, N_WORLDS, replace=True)
train_cond = generate_cond(every_conf[repeated_train_indices])

experience_replay = ()
agent_answers = ()

n_bodies = 1
action_repeat = 1
starting_step = 0
starting_episode = 0

for cond in train_cond:
    cond["timeout"] = timeout
    cond["forces"] = [[0]]


training_dfs = []
for seed in [0, 42, 72]:
    
    this_seed_model_directory = model_directory + str(seed) + "/"
    if not os.path.exists(this_seed_model_directory):
        os.makedirs(this_seed_model_directory)
    
    training_data = {"question_loss":[], "value_loss":[], "control": [], "episode_length": [], "correct_answer": []}
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    net_params = {"input_dim":10, "hidden_dim":25, "n_layers":4, "output_dim":6, "dropout":0.0}
    value_network = ValueNetwork(**net_params).to(torch.device(device))
    optimizer = optim.Adam(value_network.parameters(), lr=5e-4)
    
    agent_cond = train_cond

    trainingArgs = {"value_network": value_network, "optimizer": optimizer, 
                    "discount_factor": discount_factor, "starting_episode": starting_episode, 
                    "train_cond": agent_cond, "timeout": timeout,
                    "experience_replay": experience_replay, "agent_answers": agent_answers, 
                    "n_bodies": n_bodies, "training_data": training_data,
                    "action_repeat": action_repeat, "model_directory": this_seed_model_directory, 
                    "current_step": starting_step, "total_steps": TOTAL_STEPS, 
                    "device": device, "reward_control": True, "done_with_control": True, 
                    "reward_not_controlling_negatively": True, "remove_features_in_index": [2, 3]}
    
    training_data = train(**trainingArgs)

    df = pd.DataFrame.from_dict(training_data)
    df["seed"] = seed
    training_dfs.append(df)

control 1.00 aciertos -1.00 (UB: 0.33) eps: 1.00 done @ 51 vloss 0.279702 qloss 0.000000 accuracy 0.00:  50%|█████     | 1/2 [00:00<00:00, 30.72it/s]

N_WORLDS 2 TOTAL_STEPS 1200


control 0.50 aciertos -1.00 (UB: 0.33) eps: 1.00 done @ 600 vloss 0.139214 qloss 0.000000 accuracy 0.00: 100%|██████████| 2/2 [00:00<00:00,  6.68it/s]
control 0.50 aciertos -1.00 (UB: 0.33) eps: 1.00 done @ 600 vloss 0.018394 qloss 0.000000 accuracy 0.00: 100%|██████████| 2/2 [00:00<00:00,  3.90it/s]
control 0.00 aciertos -1.00 (UB: 0.33) eps: 1.00 done @ 600 vloss 0.014639 qloss 0.000000 accuracy 0.00: 100%|██████████| 2/2 [00:00<00:00,  3.21it/s]


In [9]:
training_dfs = pd.concat(training_dfs)
training_dfs.to_hdf(data_directory+"training_data.h5", key="training_data")

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['value_loss']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
